# Integridad Referencial




In [8]:
-- Para dejar en blanco la base de datos PRUEBA ejecute este bloque 
-- Ejecute este bloque(colapsado) para usarla  
use master

IF EXISTS  
    ( 
        SELECT name FROM master.dbo.sysdatabases  
        WHERE name = 'Pruebas' 
    ) 
    ALTER DATABASE pruebas SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE Pruebas 
GO 
    
CREATE DATABASE Pruebas 
GO 
use Pruebas


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.300

## Paso 1 ##
Sea una entidad *Empleado* que *trabaja* en un *Departamento*


In [9]:
CREATE TABLE Empleados
(
    Nombre nvarchar(30), 
    Apellido nvarchar(30),
    RFC CHAR(13), 
    CURP CHAR(18) not null primary key,
    NSS CHAR(20),
    Nacimiento DATE,
    Trabaja CHAR(3) --Lo agregamos para la relación trabaja
)

CREATE TABLE Departamentos
(
    idDepartamento CHAR(3) primary key,
    nombre NVARCHAR(30),
    descripcion NVARCHAR(200),
    mision NVARCHAR(200)
)

INSERT into Departamentos (idDepartamento, nombre) 
    values  ('VEN', 'Ventas' ), 
            ('MKT', 'Mercadotecnia'),
            ('PRO', 'Producción')

INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOEJ880305RESHS000', 'Lopez Espericueta', 'Jose', 'MKT')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LAEJ880305RESHS000', 'Larrea Espericueta', 'Jose', 'MKT')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOTJ880305RESHS000', 'Lopez Tarriba', 'Jose', 'PRO')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOEO880305RESHS000', 'Lopez Omar', 'Jose', 'VEN')            

(3 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.142

Al momento de asignar un empleado a un departamento ¿ puede hacerse aunque que este no exista ?

In [10]:
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'VICJ880305RESHS000', 'Vizcarra Camacho', 'Jaime', 'TIC')
SELECT * from Empleados


(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.039

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,TIC


## Paso 2##

La integridad referencial evita esto. Agreguemos una restriccion de llave foránea a la columna `Trabaja` de la tabla `Empleados`, para definirlo usamos la clausula `CONSTRAINT constraint_name FOREIGN KEY` que puede pertenecer a la sentencia `CREATE TABLE` o a `ALTER TABLE`
```
[ FOREIGN KEY ]
        REFERENCES [ schema_name . ] referenced_table_name [ ( ref_column ) ]
        [ ON DELETE { NO ACTION | CASCADE | SET NULL | SET DEFAULT } ]
        [ ON UPDATE { NO ACTION | CASCADE | SET NULL | SET DEFAULT } ]
        [ NOT FOR REPLICATION ]
```

In [11]:
-- Primero eliminamos la referencia al departamento inexistente
Update Empleados    
    set Trabaja = null
    where Trabaja = 'TIC'

(1 row affected)

Total execution time: 00:00:00.006

In [12]:
ALTER TABLE Empleados
    ADD CONSTRAINT fk_tabla FOREIGN KEY (trabaja) references Departamentos(idDepartamento)


Commands completed successfully.

Total execution time: 00:00:00.026

Intente agregar nuevamente un empleado con un departamento inexistente

In [13]:
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
values ('RFCA124578F3G', 'Alba','Andrea','SIS')
--Valores
SELECT * from Empleados


: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the FOREIGN KEY constraint "fk_tabla". The conflict occurred in database "Pruebas", table "dbo.Departamentos", column 'idDepartamento'.

The statement has been terminated.

(5 rows affected)

Total execution time: 00:00:00.071

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,NULL


## Paso 3: Eliminando referidos##

Asigne los empleados sin departamento al de *Ventas*

In [14]:
-- UPDATE
Update empleados set trabaja = 'VEN' where trabaja is null

(1 row affected)

Total execution time: 00:00:00.003

In [15]:
select * from empleados

(5 rows affected)

Total execution time: 00:00:00.007

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,VEN


¿Que sucede si quiero eliminar el departamento *Ventas*?

**Respuesta**

No deja porque se usa como llave foránea en la tabla empleados

Modifiquemos la restricción para que ahora al eliminar un *Departamento* elimine también a los empleados asignados a el

In [19]:
--Eliminar la restricción de llave foránea
ALTER TABLE Empleados DROP CONSTRAINT fk_tabla
 go
--No se olvide del GO

-- Agregamos la clausula ON DELETE y ON UPDATE
ALTER TABLE Empleados add CONSTRAINT fk_empleados Foreign key (trabaja) references Departamentos(idDepartamento) ON DELETE CASCADE


: Msg 2714, Level 16, State 5, Line 7
There is already an object named 'fk_empleados' in the database.

: Msg 1750, Level 16, State 0, Line 7
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.001

In [20]:
delete from departamentos where idDepartamento = 'VEN'

(1 row affected)

Total execution time: 00:00:00.021

¿Que sucede ahora si intenta eliminar el departamento *Ventas* ? 

**Respuesta**
Si deja eliminar y también elimina las tuplas que estaban relacionadas al departamento ventas



## Paso 4 ##

Modifiquemos el comportamiento de la restricción para que al eliminar un departamento los empleados queden sin uno asignado ( `SET NULL`)
¿Es necesario eliminar la restricción y volverla a crear con el nuevo comportamiento ?

**Respuesta**
No es necesario, podemos alterar la constraint directamente y se sobreescribe el comportamiento

In [21]:
ALTER TABLE Empleados DROP CONSTRAINT fk_empleados
 go

ALTER TABLE Empleados add CONSTRAINT fk_empleados Foreign key (trabaja) references Departamentos(idDepartamento) ON DELETE set null


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

Pruebe el comportamiento eliminando un departamento y muestre como queda el empleado

In [22]:
delete from departamentos where idDepartamento = 'MKT'

(1 row affected)

Total execution time: 00:00:00.017

In [23]:
select * from empleados

(3 rows affected)

Total execution time: 00:00:00.003

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,NULL
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,NULL
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO


## Comentarios a la práctica##
<Su comentario aquí>